In [132]:
import requests
import pandas as pd
from tqdm.notebook import tqdm
import time
import os
import pandas as pd
import glob

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
}

In [4]:
description = requests.get("https://showdata.gks.ru/backbone/descriptor", headers=headers).json()

In [5]:
datasets = []
no_period = 0
for obj in tqdm(description):
#     print(obj)
    if 'predefined' in obj:
        continue
    if '-17008' not in obj:
#         print(obj)
        continue
    if '-17023' not in obj:
#         print(obj)
        no_period += 1
        continue
    datasets.append({
        'id': obj['id'],
        'name': obj['-3'],
        'code': obj['-17008'],
        'period': obj['-17023'],
        'department': obj['-17022'],
        'employee': obj['-17021'],
        'last_update': obj['-17027'],
        'ehd_update': obj.get('-17058', None)
    })
#     break
print("No period cases: %s" % no_period)
datasets = pd.DataFrame(datasets)
print("Datasets count: %d (unique - %d)" % (datasets.shape[0], datasets.id.nunique()))
# datasets.to_csv("") # TODO
datasets.head()


No period cases: 76
Datasets count: 2618 (unique - 2618)


,id,name,code,period,department,employee,last_update,ehd_update
0,273992,11111110100240200001 Число торговых мест на рынке,11111110100240200001,2010 - 2020,,,2021-12-04T20:07:12+00:00,2020-11-10T22:06:15+00:00
1,275862,11111110100350200001 Наличие основных строител...,11111110100350200001,2005 - 2020,"Управление статистики строительства, инвестици...",Елисеева М.А.,2021-04-23T04:23:34+00:00,2021-04-19T06:47:57+00:00
2,273976,11111110100430200001 Количество персональных к...,11111110100430200001,2009 - 2020,"Управление статистики образования, науки и инн...",Кукушкина Н.А.,2021-10-29T03:08:13+00:00,2021-10-15T15:24:03+00:00
3,278914,"11111110100540200001 Число торговых мест, закр...",11111110100540200001,2010 - 2020,,,2021-12-04T20:21:33+00:00,2020-11-10T22:06:15+00:00
4,273636,11111110100560200001 Площадь торгового зала об...,11111110100560200001,2011 - 2019,,,2020-09-24T08:59:04+00:00,2020-09-23T22:05:36+00:00


In [ ]:
# Install selenium and download geckodriver from https://github.com/mozilla/geckodriver/releases/

In [7]:
pip install selenium

  Using cached selenium-4.1.3-py3-none-any.whl (968 kB)
  Using cached trio-0.20.0-py3-none-any.whl (359 kB)
  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached sniffio-1.2.0-py3-none-any.whl (10 kB)
  Using cached outcome-1.1.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.1.0-py3-none-any.whl (24 kB)
  Using cached pyOpenSSL-22.0.0-py2.py3-none-any.whl (55 kB)
     |████████████████████████████████| 2.5 MB 1.4 MB/s eta 0:00:01
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.7
    Uninstalling cryptography-3.4.7:
      Successfully uninstalled cryptography-3.4.7
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [26]:
!mkdir ./reports

In [117]:
options = Options()
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.dir", "%s/reports" % os.getcwd())
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/csv")
driver = webdriver.Firefox(executable_path="./geckodriver", options=options)
driver.get("https://showdata.gks.ru/")

<ipython-input-117-8031f12d8685>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path="./geckodriver", options=options)


In [118]:
processed = set()

In [169]:
TO_SKIP = set([278154, 273502]) # данные отчеты загружаются с ошибкой
for _, row in tqdm(datasets.drop_duplicates().iterrows(), total=datasets.shape[0]):
    report_id = row.id
    if report_id in processed or report_id in TO_SKIP:
        continue
    filename = "reports/%s.csv" % row['name'].replace(" ", "_")[:100]
    driver.get("https://showdata.gks.ru/report/%s" % report_id)
    time.sleep(6)
    iteration = 0
    while True:
        if iteration > 5:
            raise Exception("Unable to download %s" % report_id)
        try:
            if os.path.exists(filename):
                print("File %s already exists" % filename)
                os.remove(filename)
            driver.find_element(by=By.CSS_SELECTOR, value="div.btn-group button.dropdown-toggle").click()
            time.sleep(1)
            driver.find_element(by=By.CSS_SELECTOR, value="a.report-export-csv").click()
            time.sleep(1)
            if os.path.exists(filename):
                pd.read_csv(filename, encoding="cp1251") # check file
            else:
                candidates = list(filter(lambda x: row.code in x, glob.glob("reports/*csv")))
                if len(candidates) > 0:
                    filename = candidates[0]
                    print("Unusual filename: %s" % filename)
                    pd.read_csv(filename, encoding="cp1251") # check file
            break
        except:
            iteration += 1
            time.sleep(5)
    processed.add(report_id)

In [84]:
# пример чтения CSV
pd.read_csv("reports/11111110100240200001_Число_торговых_мест_на_рынке.csv", encoding="cp1251", skiprows=1)

,Unnamed: 0,Unnamed: 1,апрель 2019 г.,июль 2019 г.,октябрь 2019 г.,январь 2020 г.,апрель 2020 г.,июль 2020 г.,октябрь 2020 г.
0,Всего,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Сельская территория,1.АГ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Специализированный (по продаже строительных ма...,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Российская Федерация,643,177.0,208.0,208.0,208.0,271.0,39.0,39.0
4,Центральный федеральный округ,030,169.0,169.0,169.0,169.0,232.0,NaN,NaN
5,Московская область,46000000000,169.0,169.0,169.0,169.0,232.0,NaN,NaN
6,Северо-Кавказский федеральный округ,038,NaN,31.0,31.0,31.0,31.0,31.0,31.0
7,Республика Ингушетия,26000000000,NaN,31.0,31.0,31.0,31.0,31.0,31.0
8,Приволжский федеральный округ,033,8.0,8.0,8.0,8.0,8.0,8.0,8.0
9,Нижегородская область,22000000000,8.0,8.0,8.0,8.0,8.0,8.0,8.0


In [166]:
# добавляем имя файла
def get_file_name(code):
    candidates = list(filter(lambda x: code in x, glob.glob("reports/*csv")))
    assert len(candidates) < 2, code
    if len(candidates) == 0:
        return None
    return candidates[0].split("/")[1]

datasets['filename'] = datasets.code.map(get_file_name)
datasets.head()

,id,name,code,period,department,employee,last_update,ehd_update,filename
0,273992,11111110100240200001 Число торговых мест на рынке,11111110100240200001,2010 - 2020,,,2021-12-04T20:07:12+00:00,2020-11-10T22:06:15+00:00,11111110100240200001_Число_торговых_мест_на_ры...
1,275862,11111110100350200001 Наличие основных строител...,11111110100350200001,2005 - 2020,"Управление статистики строительства, инвестици...",Елисеева М.А.,2021-04-23T04:23:34+00:00,2021-04-19T06:47:57+00:00,11111110100350200001_Наличие_основных_строител...
2,273976,11111110100430200001 Количество персональных к...,11111110100430200001,2009 - 2020,"Управление статистики образования, науки и инн...",Кукушкина Н.А.,2021-10-29T03:08:13+00:00,2021-10-15T15:24:03+00:00,11111110100430200001_Количество_персональных_к...
3,278914,"11111110100540200001 Число торговых мест, закр...",11111110100540200001,2010 - 2020,,,2021-12-04T20:21:33+00:00,2020-11-10T22:06:15+00:00,"11111110100540200001_Число_торговых_мест,_закр..."
4,273636,11111110100560200001 Площадь торгового зала об...,11111110100560200001,2011 - 2019,,,2020-09-24T08:59:04+00:00,2020-09-23T22:05:36+00:00,11111110100560200001_Площадь_торгового_зала_об...


In [167]:
# случаи когда файл не обработался
datasets[datasets.filename.isnull()]

,id,name,code,period,department,employee,last_update,ehd_update,filename
21,278042,"11111131100050200001 Среднегодовая мощность, д...",11111131100050200001,2017 - 2020,,,2021-10-12T12:23:38+00:00,2021-09-28T05:27:26+00:00,None
105,279108,11111160100030200001 Общая площадь жилых помещ...,11111160100030200001,2008 - 2019,,,2021-05-21T15:08:26+00:00,2021-05-16T07:03:09+00:00,None
146,273530,11112100100230200001 Образовалось лома и отход...,11112100100230200001,2009 - 2018,,,2020-11-23T23:29:05+00:00,2019-03-30T19:51:23+00:00,None
207,273292,11211000101560200001 Среднесписочная численнос...,11211000101560200001,2016 - 2020,Управление статистики труда,Флоря Е.В.,2021-10-15T20:01:12+00:00,2021-09-07T04:46:32+00:00,None
223,278828,"11220000100200200001 Численность работников, з...",11220000100200200001,2008 - 2012,,,2021-12-31T19:55:12+00:00,None,None
...,...,...,...,...,...,...,...,...,...
2571,273894,32000000300010200001 Распределение объема прио...,32000000300010200001,2010 - 2020,,,2021-01-16T04:42:01+00:00,2021-01-15T23:06:27+00:00,None
2572,273896,32000000300020200001 Распределение объема прод...,32000000300020200001,2010 - 2020,,,2021-01-16T04:42:01+00:00,2021-01-15T23:06:27+00:00,None
2578,274284,32000000300090200001 Оценка среднего сложившег...,32000000300090200001,2010 - 2020,,,2020-12-19T22:30:58+00:00,2020-12-04T22:54:36+00:00,None
2594,273882,32000000300250200001 Оценка портфеля заказов (...,32000000300250200001,2010 - 2020,,,2021-01-16T04:35:28+00:00,2021-01-15T23:19:51+00:00,None


In [168]:
# на всякий случай перепрокачиваем кейсы, когда файл не скачался
for file in datasets[datasets.filename.isnull()].id:
#     print(file)
    try:
        processed.remove(file)
    except:
        pass

In [170]:
datasets.to_csv("data/metadata_20220419.csv", index=False)